In [1]:
import os
import datetime
import json
import twython
import json

In [2]:
import pandas as pd
import numpy as np 

In [3]:
from twython import Twython
from contextlib import suppress
from requests_oauthlib import OAuth1Session
from apscheduler.schedulers.background import BackgroundScheduler as Scheduler

In [4]:
# Enter your keys/secrets as strings in the following fields
credentials = {}
credentials['CONSUMER_KEY'] = 
credentials['CONSUMER_SECRET'] = 
credentials['ACCESS_KEY'] = 
credentials['ACCESS_SECRET'] = 

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:
    json.dump(credentials, file)

In [5]:
# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

In [6]:
# getting the header for our dataframe so we have something to append to 
startdf = pd.read_csv('df')
# when our dataframe is created the index is unnamed, when you export it is renamed so we need to fix it so our df realizes it's the same 
startdf = startdf.head(0).rename(columns={'Unnamed: 0':''})
#assign our id as the most recent tweet id to start so that we get everything after it 
id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']

In [7]:
def myfn(q, name):
    # import global variables we will be using this function to alter
    global id_
    global startdf 
    # Create our query
    query = {'q': q,
            'result_type': 'recent',
            'count': 100,
            'lang': 'en',
            'max_id': id_,
            'tweet_mode' : 'extend',
            'entities': {
                "hashtags": [],
                "symbols": [],
                "user_mentions": []
            }
            }
    #build our query into a dictionary to easily turn into dataframe 
    dict_ = {'user': [], 'user_id':[], 'post_id': [], 'text': [], 'favorite_count': [], 'hashtags':[], 'symbols':[], 'user_mentions':[], 
             'retweet_count':[]}
    for status in python_tweets.search(**query)['statuses']:
        dict_['user'].append(status['user']['screen_name'])
        dict_['user_id'].append(status['user']['id'])
        dict_['text'].append(status['text'])
        dict_['hashtags'].append(status['entities']['hashtags'])
        dict_['symbols'].append(status['entities']['symbols'])
        dict_['user_mentions'].append(status['entities']['user_mentions'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['retweet_count'].append(status['retweet_count'])
        dict_['post_id'].append(status['id'])
    # put data in a DataFrame to work with it easier
    df = pd.DataFrame(dict_)
    #removing our vairables stuck in dictionaries within our dataframe and give them their own columns
    df['mentions'] = ' '
    df['hashtag'] = ' '
    for i in range(len(df)):
        for j in range(len(df.at[i, 'user_mentions'])):
            try:
                df.at[i, 'mentions'] = str(df.at[i, 'mentions']) + ' ' + str(df.at[i, 'user_mentions'][j]['id'])
            except:
                df.at[i, 'mentions'] = str(df.at[i, 'mentions'])
        for k in range(len(df.at[i, 'hashtags'])):
            try:
                df.at[i, 'hashtag'] = str(df.at[i, 'hashtag']) + ' ' + str(df.at[i, 'hashtags'][k]['text'])
            except:
                df.at[i, 'hashtag'] = str(df.at[i, 'hashtag'])
    # get rid of columns that we took everything we need from 
    df = df.drop(['user_mentions', 'hashtags'], 1)
    startdf = startdf.append(df)
    # not sure why id_ doesn't want to change unless reset but do it because it works and can't hurt
    id_ = None
    # get oldest id_ (lowest number) and subtract one so that that you don't reinclude it in the next search
    # reset id_ to be oldest
    try: 
        id_ = int(df['post_id'].sort_values(ascending=True).to_frame().reset_index()['post_id'][0]) - 1 
    except:
        id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']
    return startdf, id_

In [8]:
# 'astronomy', 'Starship', 'mars', 'curiosityrover', 'oppertunityrover', 'starlink', 'falconheavy', 'sls','ESA', 'NASA', 'spacex', 'virgingalactic', 'virginorbit', 'JAXA', 'Roscosmos', 'artemis', 
# 'starliner', 'blueorigin', 'spacetravel'

In [9]:
query_list = ['marswebcam', 'falcon9', 'nasa_app', 'universe', 'cosmos', 'iss', 'climate', 'internationalspacestation', 'futurism', 'starliner', 'blueorigin', 'spacetravel', 'astronomy', 'Starship', 'mars', 'curiosityrover', 'oppertunityrover', 'starlink', 'falconheavy', 'sls','ESA', 'NASA', 'spacex', 'virgingalactic', 'virginorbit', 'JAXA', 'Roscosmos', 'artemis', 'areospace']

In [10]:
# eventual goal is to use those loop to do all of our searches
end_item = datetime.datetime.now()
id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']

for item in query_list:
    #reset id_ to be most recent tweet for each item
    id_ = python_tweets.search(**{'q': 'RT', 'result_type': 'recent', 'count': 2})['statuses'][0]['id']
    # for each item, reset the start time to the end of the last item 
    start_time = end_item + datetime.timedelta(seconds=1)
    # set the end time to be 90 min after you start, gathering 50 tweets every 100 seconds for 2,700 tweets from each item in list
    end_item = end_item + datetime.timedelta(seconds=1000)
    #print start and end times so I know when the program will be finished and where it should be at 
    print(f'starting {item} time : {start_time.strftime("%H:%M:%S")}, ending time: {end_item.strftime("%H:%M:%S")}')
    #occassionally getting key/value errors of 0 that I cannot find the cause of, but do not affect anything in how our dataframe
    #is made, I just don't want them printing 
    with suppress(KeyError, ValueError):
        #initiate scheduler
        sch = Scheduler()
        #add our function and items, we're doing 50/10 seconds 
        sch.add_job(myfn, 'interval', (item, startdf), seconds= 8, start_date=start_time, end_date=end_item)
        #start scheduler
        sch.start()

starting marswebcam time : 15:18:58, ending time: 15:35:37
starting falcon9 time : 15:35:38, ending time: 15:52:17
starting nasa_app time : 15:52:18, ending time: 16:08:57
starting universe time : 16:08:58, ending time: 16:25:37
starting cosmos time : 16:25:38, ending time: 16:42:17
starting iss time : 16:42:18, ending time: 16:58:57
starting climate time : 16:58:58, ending time: 17:15:37
starting internationalspacestation time : 17:15:38, ending time: 17:32:17
starting futurism time : 17:32:18, ending time: 17:48:57
starting starliner time : 17:48:58, ending time: 18:05:37
starting blueorigin time : 18:05:38, ending time: 18:22:17
starting spacetravel time : 18:22:18, ending time: 18:38:57
starting astronomy time : 18:38:58, ending time: 18:55:37
starting Starship time : 18:55:38, ending time: 19:12:17
starting mars time : 19:12:18, ending time: 19:28:57
starting curiosityrover time : 19:28:58, ending time: 19:45:37
starting oppertunityrover time : 19:45:38, ending time: 20:02:17
star

In [13]:
len(startdf)

248100

In [ ]:
# df = pd.read_csv('alltweets39')

In [ ]:
# newstartdf = startdf[['user', 'user_id', 'text', 'favorite_count', 'retweet_count',
#                    'mentions', 'hashtag', 'post_id']].copy()

In [ ]:
# df = df.append(newstartdf)
# df = df.drop_duplicates(subset='post_id')
# df = df.sort_values(by='favorite_count', ascending=False).drop_duplicates(subset='text', keep='first')
# df = df.reset_index()
# print(len(df))
# #171476

In [ ]:
# df['hashtag'].value_counts().to_frame().head(50)

In [14]:
# startdf.to_csv('beeep')

Run time of job "myfn (trigger: interval[0:00:08], next run at: 2021-03-20 22:48:58 PDT)" was missed by 11:03:13.538887
Run time of job "myfn (trigger: interval[0:00:08], next run at: 2021-03-20 23:05:38 PDT)" was missed by 11:03:13.541578


In [ ]:
# since every time we loop through it starts a new index, there are now multiple entries for each index, we need to fix this
# and the easiest way is to just reset the index once it finishes running 
startdf = startdf.drop_duplicates(subset='post_id')
startdf = startdf.reset_index()

In [ ]:
# don't need these columns anymore
startdf = startdf.drop(['index', '', 'symbols'], 1)

In [ ]:
# in order to add our strings for our hashtags and mentions together we had to make the entire column a string, sinlucding empty cells
# here we go through and replace empty cells with nan values so pandas will read them as being empty instead of a string
for i in range(len(startdf)):
    if startdf.at[i, 'mentions'] == ' ':
        startdf.at[i, 'mentions'] = np.nan
    else: 
        pass 
    if startdf.at[i, 'hashtag'] == ' ':
        startdf.at[i, 'hashtag'] = np.nan
    else: 
        pass 